In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# tf.enable_eager_execution()
print(tf.__version__)

1.15.0


In [3]:
users = ['Ryan', 'Daniell', 'Vijay', 'Chris']
movies = ['Star Wars', 'The Dark Knight', 'Shrek', 'The Incredibles', 'Bleu', 'Memento']
features = ['Action', 'Sci-Fi', 'Comedy', 'Cartoon' ,'Drama']

num_users = len(users)
num_movies = len(movies)
num_feats = len(features)

In [4]:
users_movies = tf.constant([
                [4, 6, 8, 0, 0],
                [0, 0, 10, 0, 8],
                [0, 6, 0, 0, 3],
                [10, 9, 0, 5, 0]
    ], dtype=tf.float32)

In [5]:
movies_feats = tf.constant([
            [1, 1, 0, 0, 1],
            [1, 1, 0, 0, 0],
            [0, 0, 1, 1, 0],
            [1, 0, 1, 1, 0],
            [0, 0, 0, 0, 1]
], dtype=tf.float32)

In [29]:
# each row represents a user's rating for the different movies
users_movies = tf.constant([
                [4,  6,  8,  0, 0, 0],
                [0,  0, 10,  0, 8, 3],
                [0,  6,  0,  0, 3, 7],
                [10, 9,  0,  5, 0, 2]],dtype=tf.float32)

# features of the movies one-hot encoded
# e.g. columns could represent ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
movies_feats = tf.constant([
                [1, 1, 0, 0, 1],
                [1, 1, 0, 0, 0],
                [0, 0, 1, 1, 0],
                [1, 0, 1, 1, 0],
                [0, 0, 0, 0, 1],
                [1, 0, 0, 0, 1]],dtype=tf.float32)

In [30]:
wgtd_feature_matrices = [tf.expand_dims(tf.transpose(users_movies)[:, i], axis=1) * movies_feats for i in range(num_users)]

In [31]:
wgtd_feature_matrices

[<tf.Tensor 'mul_9:0' shape=(6, 5) dtype=float32>,
 <tf.Tensor 'mul_10:0' shape=(6, 5) dtype=float32>,
 <tf.Tensor 'mul_11:0' shape=(6, 5) dtype=float32>,
 <tf.Tensor 'mul_12:0' shape=(6, 5) dtype=float32>]

In [7]:
users_movies_feats = tf.stack(wgtd_feature_matrices, axis=0)

In [8]:
users_movies_feats

<tf.Tensor 'stack:0' shape=(4, 5, 5) dtype=float32>

In [9]:
users_movies_feats_sums = tf.reduce_sum(users_movies_feats, axis=1)
users_movies_feats_totals = tf.reduce_sum(users_movies_feats_sums, axis=1)

In [10]:
users_feats = tf.stack([users_movies_feats_sums[i, :] / users_movies_feats_totals[i] for i in range(num_users)], axis=0)

In [32]:
users_feats = users_feats/tf.reduce_sum(users_feats,axis=1,keepdims=True)
users_feats

<tf.Tensor 'truediv_4:0' shape=(4, 5) dtype=float32>

In [34]:
users_feats

<tf.Tensor 'truediv_4:0' shape=(4, 5) dtype=float32>

In [36]:
top_users_features = tf.nn.top_k(users_feats, num_feats)[1]
top_users_features

<tf.Tensor 'TopKV2_4:1' shape=(4, 5) dtype=int32>

In [11]:
def find_user_top_feats(user_index):
    feats_ind = tf.nn.top_k(users_feats[user_index], num_feats)[1]
    return tf.gather_nd(features, tf.expand_dims(feats_ind, axis=1))

In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    users_topfeats = {}
    for i in range(num_users):
        top_feats = sess.run(find_user_top_feats(i))
#         print(top_feats)
        for j in range(len(top_feats)):
            top_feats[j] = top_feats[j].decode('utf-8')
        print(top_feats)
        users_topfeats[users[i]] = list(top_feats)

['Action' 'Sci-Fi' 'Comedy' 'Cartoon' 'Drama']
['Comedy' 'Cartoon' 'Drama' 'Action' 'Sci-Fi']
['Action' 'Sci-Fi' 'Drama' 'Comedy' 'Cartoon']
['Action' 'Sci-Fi' 'Drama' 'Comedy' 'Cartoon']


In [13]:
users_topfeats

{'Ryan': ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama'],
 'Daniell': ['Comedy', 'Cartoon', 'Drama', 'Action', 'Sci-Fi'],
 'Vijay': ['Action', 'Sci-Fi', 'Drama', 'Comedy', 'Cartoon'],
 'Chris': ['Action', 'Sci-Fi', 'Drama', 'Comedy', 'Cartoon']}

In [14]:
users_feats[0]

<tf.Tensor 'strided_slice_16:0' shape=(5,) dtype=float32>

In [15]:
users_ratings = [tf.map_fn(lambda x: tf.tensordot(users_feats[i], x, axes=1), movies_feats) for i in range(num_users)]

In [16]:
users_ratings

[<tf.Tensor 'map/TensorArrayStack/TensorArrayGatherV3:0' shape=(5,) dtype=float32>,
 <tf.Tensor 'map_1/TensorArrayStack/TensorArrayGatherV3:0' shape=(5,) dtype=float32>,
 <tf.Tensor 'map_2/TensorArrayStack/TensorArrayGatherV3:0' shape=(5,) dtype=float32>,
 <tf.Tensor 'map_3/TensorArrayStack/TensorArrayGatherV3:0' shape=(5,) dtype=float32>]

In [20]:
users_ratings

[<tf.Tensor 'map/TensorArrayStack/TensorArrayGatherV3:0' shape=(5,) dtype=float32>,
 <tf.Tensor 'map_1/TensorArrayStack/TensorArrayGatherV3:0' shape=(5,) dtype=float32>,
 <tf.Tensor 'map_2/TensorArrayStack/TensorArrayGatherV3:0' shape=(5,) dtype=float32>,
 <tf.Tensor 'map_3/TensorArrayStack/TensorArrayGatherV3:0' shape=(5,) dtype=float32>]

In [17]:
all_users_ratings = tf.stack(users_ratings)

In [24]:
all_users_ratings[0]

<tf.Tensor 'strided_slice_18:0' shape=(5,) dtype=float32>

In [18]:
all_users_ratings_new = tf.where(tf.equal(users_movies, tf.zeros_like(users_movies)),
                                all_users_ratings,
                                -np.inf*tf.ones_like(tf.cast(users_movies, tf.float32)))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [39]:
def find_user_top_movies(user_index, num_to_recommend):
    m_ind = tf.nn.top_k(all_users_ratings_new[user_index], num_to_recommend)[1]
    return tf.gather_nd(movies, tf.expand_dims(m_ind, axis=1))

In [40]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    user_topmovies = {}
    num_to_recommend = tf.reduce_sum(tf.cast(tf.equal(users_movies, tf.zeros_like(users_movies)), dtype=tf.float32), axis=1)
    for index in range(num_users):
        top_movies = sess.run(find_user_top_movies(index, tf.cast(num_to_recommend[index], dtype=tf.int32)))
        user_topmovies[users[index]] = list(top_movies)

In [41]:
user_topmovies

{'Ryan': [b'The Incredibles', b'Bleu', b'Star Wars'],
 'Daniell': [b'The Incredibles', b'Star Wars', b'The Dark Knight'],
 'Vijay': [b'Star Wars', b'The Incredibles', b'Shrek'],
 'Chris': [b'Shrek', b'Bleu']}